# Unscented Kalman Filter

## Introduction
In this example we show how to implement a particle filter. As an example we will use the model of a CSTR reactor. 

<img src="../images/bioreactor.png" alt="drawing" width="600"/>

The model is 

$$
\begin{align}\label{eq:temp}
\dot{x}_1(t) &= -a_1 x_1(t) + b_1r  \\
\dot{x}_2(t) &= -a_2 x_2(t) + b_2r + gu \\
\end{align}
$$

where $r = (1 - x_1(t))\exp(-E/(1 + x_2(t)))$ where $a_1,b_1,a_2,b_2,E$ and $g$ are constants. The goal is estimating the concentrations of $x_1$ and $x_2$ from noisy measurements.


## Code

In [1]:
import numpy as np

from hilo_mpc import UKF, set_plot_backend
from hilo_mpc.library import cstr_schaffner_and_zeitz as cstr

# Seed for noise
np.random.seed(0)
set_plot_backend('bokeh')

In [2]:
model = cstr()

# Set up model
model.setup(dt=.1)

In [3]:
# Initialize Kalman filter
ukf = UKF(model, plot_backend='bokeh', alpha=1, beta=0, kappa=1)

# Set up Kalman filter
ukf.setup()


In [4]:
# Constants
const = {
    'a_1': .2674,
    'a_2': 1.815,
    'b_1': 1.05e14,
    'b_2': 4.92e13,
    'g': 1.5476,
    'E': 34.2583
}


# Initial conditions of the model
model.set_initial_conditions([.5, 0.])
model.set_initial_parameter_values(const)

# Initial guess for the Kalman filter
ukf.R = 1e-4
ukf.set_initial_guess([0., 0.], P0=[.25, .25])
ukf.set_initial_parameter_values(const)

In [5]:
# Run simulations
for _ in range(200):
    model.simulate(u=-.002)

    # Get noisy measurement and calculate estimates
    yk = model.solution.make_some_noise('y:f', var={'y': 1e-4})
    ukf.estimate(y=yk, u=-.002)

## Plots

In [7]:
model.solution.plot(
    ('t', 'x_1'),
    ('t', 'x_2'),
    ('t', 'y', 'y_noisy'),
    data=ukf.solution,
    data_suffix='pred',
    data_skip=2,
    title=("conversion of chemical reaction", "scaled reactor temperature", "measured temperature"),
    output_file='ukf_cstr/actual_vs_predicted.html'
)
ukf.solution.plot(('t', 'P_0', 'P_3'), output_file='ukf_cstr/error_variance.html')

'ukf_cstr/error_variance.html'